In [24]:
from pathlib import Path
from tempfile import NamedTemporaryFile
import os

import numba
import numpy as np
from mol2vec import features
from rdkit import Chem
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import AllChem
from gensim.models import word2vec

In [2]:
model = word2vec.Word2Vec.load("mol2vec_model.pkl")

In [3]:
print(len(model.wv.vocab))

8062


In [4]:
# Generate some small molecules for comparison and
# sanity check!

benzene = Chem.MolFromSmiles("c1ccccc1")
acetaldehyde = Chem.MolFromSmiles("CC=O")
benzonitrile = Chem.MolFromSmiles("C1=CC=C(C=C1)C#N")
phenol = Chem.MolFromSmiles("Oc1ccccc1")
hc5n = Chem.MolFromSmiles("C#CC#CC#N")

molecules = [benzene, acetaldehyde, benzonitrile, phenol, hc5n]

We now convert the `Mol` objects into "sentences" to be processed by our `mol2vec` model.

In [5]:
# Translate molecules
sentences = [features.mol2alt_sentence(mol, 1) for mol in molecules]

# Use our trained model to generate vectors for every sentence
vectors = features.sentences2vec(sentences, model)

So now every molecule is converted into a 300 dimensional vector! To check how similar molecules are, we can compute the cosine similarity, which is given by this formula:

$$ \cos(\theta) = \frac{\bf{A} \dot \bf{B}}{\vert\vert \bf{A} \vert\vert \vert\vert \bf{B} \vert\vert} $$

This metric basically measures the angle subtended between two vectors $\bf{A}, \bf{B}$. For orthogonal vectors (i.e. completely dissimilar), $\cos(\theta)=0$ and two equivalent vectors $\cos(\theta)=1$ (two equivalent molecules). The code below implements this measure, as well as a fast(er) implementation for calculating every pair.

In [6]:
@numba.jit(fastmath=True)
def cosine_similarity(A, B):
    return np.dot(A, B) / (np.linalg.norm(A) * np.linalg.norm(B))


@numba.jit(fastmath=True)
def pairwise_similarity(vectors):
    n = len(vectors)
    matrix = np.zeros((n, n), dtype=np.float32)
    for i in range(n):
        for j in range(n):
            matrix[i,j] = cosine_similarity(vectors[i], vectors[j])
    return matrix

In [7]:
pairwise_similarity(vectors)

array([[1.        , 0.3647529 , 0.92900187, 0.9687872 , 0.20026813],
       [0.3647529 , 1.        , 0.44702974, 0.4217399 , 0.37271535],
       [0.92900187, 0.44702974, 0.9999999 , 0.9402652 , 0.44940293],
       [0.9687872 , 0.4217399 , 0.9402652 , 1.        , 0.22655925],
       [0.20026813, 0.37271535, 0.44940293, 0.22655925, 1.        ]],
      dtype=float32)

This matrix shows the cosine similarity between every molecule pair: the diagonal entries are all 1, because they're the same molecule (good sanity check), while the off-diagonal elements compare each molecule.

In row order, we have: benzene, acetaldehyde, benzonitrile, phenol, HC5N. From chemical intuition, benzene, benzonitrile, and phenol should be very similar with one another because they all contain a benzene ring. Acetaldehyde should be closest to phenol because of oxygen. HC5N should be similar to benzonitrile because of the nitrile group.

The first and third statements hold well, and they meet our expectations! The second one is a bit more dicey, as the result suggests that acetaldehyde is closer to benzonitrile than phenol despite what we originally thought. In this case, it's very likely that our `mol2vec` model isn't entirely doing what we're thinking, and we would need to do more tests to understand why this is. The encoding must not be focusing on the oxygen as much: for these comparisons, we would have to either: (a) tune the `mol2vec` model, (b) use a different type of encoding, or (c) use a different measure of similarlity.

In [9]:
with open("collected_smiles.smi") as read_file:
    smi_list = read_file.readlines()

In [16]:
smi_list = [smi.strip() for smi in smi_list]

_The step below takes a long time!_ Should parallelize it.

In [27]:
sentences = list()
for smi in smi_list:
    mol = Chem.MolFromSmiles(smi, sanitize=False)
    mol.UpdatePropertyCache(strict=False)
    Chem.GetSymmSSSR(mol)
    sentence = features.mol2alt_sentence(mol, 1)
    sentences.append(sentence)

In [28]:
# Use our trained model to generate vectors for every sentence
vectors = features.sentences2vec(sentences, model)

In [59]:
np.savez("embeddings.npz", vectors)